In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

In [70]:
def strip_tags(soup, tags):
    for tag in tags:
        for match in soup.findAll(tag):
            match.replaceWithChildren()

soup = BeautifulSoup("""
    <td>
    <div align="center">W. Force</div>
    </td>
    <td>
    <div align="center">5</div>
    </td>
    <td>
    <div align="center">18/2/06</div>
    </td>
    """, 'html.parser' )          
strip_tag(soup, ["div", "invalid"])
print(soup)


<td>
W. Force
</td>
<td>
5
</td>
<td>
18/2/06
</td>



In [143]:
teams_1 = []
teams_2 = []
scores_1 = []
scores_2 = []
dates = []
for year in [str(y) for y in range(2006, 2018)]:
    url = 'http://www.superxv.com/results/{year}-super-rugby-results/'.format(year=year)
    page = requests.get(url)
    if page.status_code != 200:
        print('Error downloading {url}'.format(url=url))
    else:
        print('Successfully downloaded {url}'.format(url=url))
        for table in BeautifulSoup(page.content, 'html.parser').find_all('table'):
            strip_tags(table, ["div"])
            for row in table.find_all('tr')[1:]:
                entries = row.find_all('td')
                if entries[0].text.strip() not in ['Bye', 'Not playing']: # and entries[1] = '':
                    teams_1.append(entries[0].text.strip().replace('\n',' '))
                    teams_2.append(entries[3].text.strip().replace('\n',' '))
                    scores_1.append(entries[1].text.strip())
                    scores_2.append(entries[4].text.strip())
                    dates.append(entries[5].text.strip())

Successfully downloaded http://www.superxv.com/results/2006-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2007-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2008-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2009-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2010-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2011-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2012-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2013-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2014-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2015-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2016-super-rugby-results/
Successfully downloaded http://www.superxv.com/results/2017-super-rugby-results/


In [162]:
########### Cleaning and formatting dates

df = pd.DataFrame({
        'date':dates,
        'team_1':teams_1,
        'team_2':teams_2,
        'score_1':scores_1,
        'score_2':scores_2})
df.date[df.date == "29/3/088"] = "29/03/2008"
df.date[df.date == "00/05/09"] = "09/05/09"
df.date[df.date == "0907/16"] = "09/07/16"
df.date = pd.to_datetime(df.date)

array(['Blues', 'Force', 'Cheetahs', 'Crusaders', 'Reds', 'Cats', 'Sharks',
       'Highlanders', 'Bulls', 'Hurricanes', 'Stormers', '', 'Chiefs',
       'Brumbies', 'Waratahs', 'Lions', 'W. Force', 'W Force',
       'Melbourne\xa0Rebels', 'Western\xa0Force', 'Melbourne Rebels',
       'Western Force', 'S.Kings', 'Sunwolves', 'Southern Kings',
       'Jaguares', 'Southern\xa0Kings'], dtype=object)